In [1]:
import dask
import dask.threaded
import dask.multiprocessing
from dask.distributed import Client

c = Client()


/home/alberta/anaconda2/envs/pangeo-cloud/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/alberta/anaconda2/envs/pangeo-cloud/lib/python3.6/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [2]:
c

Client Scheduler: tcp://127.0.0.1:38701 Dashboard: http://127.0.0.1:43067/status,Cluster Workers: 4 Cores: 8 Memory: 33.66 GB


In [3]:
import sys, glob
import numpy as np
import xarray as xr
import xscale.spectral.fft as xfft
import xscale 
import Wavenum_freq_spec_func as wfs
import time
import zarr


In [4]:
dsu=xr.open_zarr('/mnt/alberta/equipes/IGE/meom/workdir/albert/eNATL60/zarr/eNATL60-BLB002-SSU-1h')


In [5]:
dsu

<xarray.Dataset>
Dimensions:              (axis_nbounds: 2, time_counter: 8760, x: 8354, y: 4729)
Coordinates:
    nav_lat              (y, x) float32 dask.array<shape=(4729, 8354), chunksize=(296, 1045)>
    nav_lon              (y, x) float32 dask.array<shape=(4729, 8354), chunksize=(296, 1045)>
    time_centered        (time_counter) datetime64[ns] dask.array<shape=(8760,), chunksize=(744,)>
  * time_counter         (time_counter) datetime64[ns] 2009-07-01T00:30:00 ... 2009-07-30T23:30:00
Dimensions without coordinates: axis_nbounds, x, y
Data variables:
    sozocrtx             (time_counter, y, x) float32 dask.array<shape=(8760, 4729, 8354), chunksize=(24, 120, 120)>
    time_counter_bounds  (time_counter, axis_nbounds) datetime64[ns] dask.array<shape=(8760, 2), chunksize=(744, 2)>
Attributes:
    Conventions:  CF-1.6
    NCO:          4.4.6
    TimeStamp:    07/01/2019 17:04:54 +0100
    description:  ocean U grid variables
    file_name:    eNATL60-BLB002_1h_20090630_20090704_gr

In [6]:
%%time
lat=dsu['nav_lat']
lon=dsu['nav_lon']
 
latmin = 40.0; latmax = 45.0;
lonmin = -40.0; lonmax = -35.0;

domain = (lonmin<lon) * (lon<lonmax) * (latmin<lat) * (lat<latmax)
where = np.where(domain)

#get indice
jmin = np.min(where[0][:])
jmax = np.max(where[0][:])
imin = np.min(where[1][:])
imax = np.max(where[1][:])

latbox=lat[jmin:jmax,imin:imax]
lonbox=lon[jmin:jmax,imin:imax]


CPU times: user 3.25 s, sys: 120 ms, total: 3.37 s
Wall time: 6.81 s


In [8]:
for month in np.arange(1,13,1):
    print('month nb '+str(month))
    print('index from '+str((month-1)*24*30)+' to '+str((month)*24*30))
    u_month=dsu['sozocrtx'][(month-1)*24*30:(month)*24*30,jmin:jmax,imin:imax]
    check_NaN = (1*np.isnan(u_month.values)).sum()
    print(check_NaN)
    

month nb 1
index from 0 to 720
0
month nb 2
index from 720 to 1440
0
month nb 3
index from 1440 to 2160
0
month nb 4
index from 2160 to 2880
0
month nb 5
index from 2880 to 3600
0
month nb 6
index from 3600 to 4320
0
month nb 7
index from 4320 to 5040
0
month nb 8
index from 5040 to 5760
0
month nb 9
index from 5760 to 6480
11987928
month nb 10
index from 6480 to 7200
0
month nb 11
index from 7200 to 7920
0
month nb 12
index from 7920 to 8640
0
